# 실전 튜닝 전략 A/B 테스트 워크숍
## 1회차: 데이터·포맷이 성능을 어떻게 바꾸는가

### 워크숍 개요
- **목표**: 동일 모델/LoRA 설정에서 데이터 품질과 포맷이 SLM 파인튜닝 성능에 미치는 영향을 A/B 테스트로 체감한다
- **소요시간**: 약 3시간
- **환경**: Google Colab (T4 GPU)
- **베이스 모델**: LGAI-EXAONE/EXAONE-4.0-1.2B (LG AI Research, 한국어 특화)

### 학습 목표
1. Instruction 튜닝 데이터의 품질이 모델 성능에 미치는 영향을 이해한다
2. Chat template과 데이터 포맷의 중요성을 실험으로 확인한다
3. Teacher-student distillation 관점에서 데이터 설계 전략을 익힌다

## 0. 환경 설정

> ⚠️ **런타임 설정 확인**: `런타임` → `런타임 유형 변경` → **T4 GPU** 선택 후 저장

In [ ]:
# GPU 확인
!nvidia-smi

Mon Feb 23 00:07:49 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.82.07              Driver Version: 580.82.07      CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   65C    P8             13W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
%%capture
# 필수 라이브러리 설치 (약 2-3분 소요)
# 참고: bitsandbytes는 paged_adamw_8bit 옵티마이저용으로 설치합니다.
#       1.2B 모델은 fp16으로 충분하므로 모델 양자화(4-bit)는 사용하지 않습니다.
!pip install -U transformers accelerate peft bitsandbytes trl datasets

In [ ]:
import torch
import json
import random
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    EarlyStoppingCallback,
)
from peft import LoraConfig, get_peft_model, PeftModel
from trl import SFTTrainer, SFTConfig
from datasets import Dataset

# 시드 고정
def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

seed_everything(42)

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_mem / 1024**3:.1f} GB")

PyTorch version: 2.10.0+cu128
CUDA available: True
GPU: Tesla T4


AttributeError: 'torch._C._CudaDeviceProperties' object has no attribute 'total_mem'

---
## 오늘 실습의 주요 포인트!

### 데이터 품질이 SFT에서 가장 큰 영향을 끼친다!

SLM 파인튜닝에서 가장 흔한 오해 중 하나는 "모델을 바꾸거나 하이퍼파라미터를 조정하면 성능이 크게 오른다"는 것입니다.  
실제로는 **동일 모델/LoRA 설정에서 데이터 품질이 가장 큰 성능 차이를 만듭니다.**

### Instruction 튜닝의 포맷

Instruction 튜닝 데이터는 보통 다음과 같은 스키마를 따릅니다:
```json
{
  "instruction": "사용자의 질문 또는 지시",
  "input": "(선택) 추가 컨텍스트",
  "output": "모델이 생성해야 할 답변"
}
```

여기서 중요한 것은 **chat template**입니다. 학습 시 사용한 포맷과 추론 시 사용하는 포맷이 일치해야 모델이 제대로 작동합니다.

### Teacher-Student Distillation 관점 (Dataset Distillation)

- Teacher 모델(예: GPT-5)의 출력을 Student 모델(예: EXAONE 1.2B)의 학습 데이터로 사용
- Teacher output의 **스타일, 구조, 난이도**가 Student의 최종 성능을 크게 좌우
- 같은 질문이라도 "짧고 두리뭉실한 답" vs "체계적이고 논리적인 답"은 완전히 다른 모델을 만듦

### 모델 크기별 권장 튜닝 전략 (T4 16GB 기준)

QLoRA(4-bit 양자화 + LoRA)는 GPU 메모리가 부족할 때 사용하는 기법입니다.  
그런데 **1~2B 크기의 소형 모델은 fp16으로도 T4 GPU(16GB)에 충분히 올라갑니다.**

작은 모델에 4-bit 양자화를 적용하면 상대적 정보 손실이 커서, 학습은 되지만(loss는 내려가지만) **생성 품질이 크게 떨어지는 현상**이 자주 발생합니다.

| 모델 크기 | GPU 메모리 (fp16) | 권장 전략 | 비고 |
|-----------|-------------------|-----------|------|
| **0.6B~1.5B** | **~3GB** | **Full SFT (fp16)** 또는 **fp16 + LoRA** | 양자화 불필요, 메모리 충분 |
| 3B~4B | ~8GB | fp16 + LoRA | Full SFT도 가능하지만 빠듯할 수 있음 |
| 7B~8B | ~16GB | QLoRA (4-bit + LoRA) | 이때부터 양자화가 필요 |
| 13B+ | 16GB 초과 | QLoRA 필수 또는 멀티 GPU | T4 1장으로 Full SFT 불가 |

> 💡 **이번 실습에서는 EXAONE 4.0-1.2B(~2.4GB)를 fp16 + LoRA로 학습합니다.** 양자화 없이 모델의 표현력을 최대한 유지하면서 LoRA로 효율적으로 튜닝하는 방식입니다.

---
## 실습 1: 데이터 버전 A/B 만들기

"여행 추천" 도메인에서 같은 질문에 대해 서로 다른 품질의 답변 데이터를 구축합니다.

| | 버전 A (Low Quality) | 버전 B (High Quality) |
|---|---|---|
| **답변 길이** | 1-2문장, 짧음 | 3-5문장, 구조적 |
| **Reasoning** | 거의 없음 | 단계적 (문제파악→접근→답) |
| **역할 부여** | 없음 | "당신은 전문 여행 컨설턴트입니다" |
| **포맷 통일** | 비일관적 | 일관된 구조 |

In [ ]:
# =============================================================
# 버전 A: Low Quality 데이터 (짧은 답변, reasoning 부재, 비일관적 포맷)
# =============================================================
version_a_data = [
    {"instruction": "제주도 여행 추천해줘", "input": "", "output": "제주도는 좋아요. 한라산이랑 성산일출봉 가보세요."},
    {"instruction": "겨울에 갈만한 국내 여행지?", "input": "", "output": "강원도 가면 됩니다. 스키장 많아요."},
    {"instruction": "2박3일 부산 여행 코스 짜줘", "input": "", "output": "해운대, 광안리, 감천문화마을 가세요."},
    {"instruction": "유럽 여행 처음인데 어디가 좋을까?", "input": "", "output": "파리 가세요. 에펠탑 보면 됩니다."},
    {"instruction": "가족 여행으로 동남아 추천해줘", "input": "", "output": "태국 가세요. 아이들이 좋아해요."},
    {"instruction": "혼자 여행하기 좋은 곳 추천", "input": "", "output": "일본 추천합니다. 혼밥 문화가 잘 되어있어요."},
    {"instruction": "여행 가방 꾸리는 팁 알려줘", "input": "", "output": "옷 돌돌 말아서 넣으세요."},
    {"instruction": "해외여행 보험 필요해?", "input": "", "output": "네 들어가세요. 필수입니다."},
    {"instruction": "비행기 싼 거 어떻게 구해?", "input": "", "output": "스카이스캐너에서 찾으세요."},
    {"instruction": "여름에 시원한 여행지 추천", "input": "", "output": "강릉이나 속초 가세요. 바다 있어요."},
    {"instruction": "신혼여행 어디로 갈까?", "input": "", "output": "몰디브요. 좋습니다."},
    {"instruction": "일본 온천 여행 추천해줘", "input": "", "output": "벳부 가세요. 온천 유명해요."},
    {"instruction": "배낭여행 준비물 알려줘", "input": "", "output": "배낭, 여권, 옷 챙기세요."},
    {"instruction": "아이와 함께 갈 만한 국내 여행지", "input": "", "output": "에버랜드 가세요."},
    {"instruction": "제주도 맛집 추천해줘", "input": "", "output": "흑돼지 먹으세요. 맛있어요."},
    {"instruction": "여행 중 환전 어떻게 해?", "input": "", "output": "공항에서 하면 됩니다."},
    {"instruction": "캠핑 처음인데 뭐 준비해야 해?", "input": "", "output": "텐트랑 침낭 필요합니다."},
    {"instruction": "서울 관광 코스 추천해줘", "input": "", "output": "경복궁이랑 명동 가세요."},
    {"instruction": "비 오는 날 여행지 추천", "input": "", "output": "실내 관광지 가세요. 박물관 같은 데."},
    {"instruction": "해외에서 인터넷 어떻게 써?", "input": "", "output": "포켓와이파이 빌리세요."},
    {"instruction": "베트남 다낭 여행 코스", "input": "", "output": "바나힐이랑 미케비치 가세요."},
    {"instruction": "여행지에서 사진 잘 찍는 법", "input": "", "output": "역광 피하고 찍으세요."},
    {"instruction": "가을 단풍 여행지 추천", "input": "", "output": "설악산이나 내장산 가세요."},
    {"instruction": "호캉스 추천해줘", "input": "", "output": "서울 5성급 호텔 가세요."},
    {"instruction": "렌터카 여행 팁", "input": "", "output": "네비 꼭 쓰세요."},
    {"instruction": "여행 예산 세우는 법", "input": "", "output": "항공, 숙소, 식비 나눠서 계산하세요."},
    {"instruction": "스위스 여행 추천 코스", "input": "", "output": "인터라켄 가세요. 경치 좋아요."},
    {"instruction": "크루즈 여행 어때?", "input": "", "output": "좋아요. 배멀미만 조심하세요."},
    {"instruction": "반려동물과 여행할 수 있는 곳", "input": "", "output": "펜션 찾아보세요."},
    {"instruction": "워케이션 좋은 곳 추천", "input": "", "output": "제주도나 양양 가세요. 카페 많아요."},
]

print(f"Version A 데이터 수: {len(version_a_data)}")
print(f"\n[예시] Version A 첫 번째 데이터:")
print(json.dumps(version_a_data[0], ensure_ascii=False, indent=2))

In [ ]:
# =============================================================
# 버전 B: High Quality 데이터 (체계적 reasoning, 통일된 스타일, 역할 부여)
# =============================================================
version_b_data = [
    {
        "instruction": "제주도 여행 추천해줘",
        "input": "",
        "output": "당신은 전문 여행 컨설턴트입니다.\n\n**1. 상황 파악**: 제주도는 사계절 매력이 다른 대한민국 대표 관광지입니다.\n\n**2. 추천 코스**:\n- 자연 탐방: 한라산 등반(성판악 코스 약 4시간), 성산일출봉(유네스코 세계자연유산)\n- 해안 드라이브: 용머리해안 → 중문관광단지 → 주상절리대\n- 문화 체험: 제주 돌문화공원, 오설록 티뮤지엄\n\n**3. 실용 팁**: 렌터카가 필수이며, 동쪽(성산·우도)과 서쪽(협재·한림)을 나눠 일정을 짜면 이동 피로를 줄일 수 있습니다. 숙소는 중문이나 서귀포 시내를 베이스캠프로 추천합니다."
    },
    {
        "instruction": "겨울에 갈만한 국내 여행지?",
        "input": "",
        "output": "당신은 전문 여행 컨설턴트입니다.\n\n**1. 상황 파악**: 겨울 국내 여행은 크게 '스키·눈 체험형'과 '따뜻한 남쪽 힐링형'으로 나뉩니다.\n\n**2. 추천 여행지**:\n- 스키·눈 체험: 강원도 평창(알펜시아·용평 리조트), 하이원 리조트 – 스키 시즌은 12월~3월 초\n- 겨울 바다: 부산 해운대 겨울 바다, 여수 낭만포차거리 – 비수기라 숙소가 저렴\n- 온천 힐링: 충남 아산 온천, 부산 동래 온천 – 노천탕에서 겨울 경치 감상\n\n**3. 실용 팁**: 강원도 방면은 주말 교통 정체가 심하므로 금요일 저녁 출발 또는 KTX 이용을 추천합니다. 겨울 바다 여행은 방풍 자켓과 핫팩을 꼭 챙기세요."
    },
    {
        "instruction": "2박3일 부산 여행 코스 짜줘",
        "input": "",
        "output": "당신은 전문 여행 컨설턴트입니다.\n\n**1. 상황 파악**: 2박 3일이면 부산의 핵심 명소를 효율적으로 둘러볼 수 있습니다.\n\n**2. 추천 코스**:\n- **Day 1** (해안 코스): 해운대 해수욕장 → 더베이101(점심) → 해동용궁사 → 광안리(야경)\n- **Day 2** (문화 코스): 감천문화마을(오전) → 국제시장·BIFF광장(점심·간식) → 영도 흰여울마을 → 자갈치시장(저녁 회)\n- **Day 3** (자연 코스): 태종대(오전) → 송도 해상 케이블카 → 서면(쇼핑·카페)\n\n**3. 실용 팁**: 부산은 지하철과 버스 연계가 좋아 대중교통만으로 충분합니다. '비짓부산패스'를 구매하면 주요 관광지 입장료와 교통을 묶어서 절약할 수 있습니다."
    },
    {
        "instruction": "유럽 여행 처음인데 어디가 좋을까?",
        "input": "",
        "output": "당신은 전문 여행 컨설턴트입니다.\n\n**1. 상황 파악**: 유럽 첫 여행이라면 대중교통이 편리하고 관광 인프라가 잘 갖춰진 도시가 적합합니다.\n\n**2. 추천 루트** (10일 기준):\n- 파리(3일): 에펠탑, 루브르 박물관, 몽마르뜨 – 유럽 여행의 클래식 시작점\n- 스위스 인터라켄(2일): 융프라우, 호수 유람 – 자연경관의 하이라이트\n- 로마(3일): 콜로세움, 바티칸, 트레비 분수 – 역사와 미식\n- 바르셀로나(2일): 사그라다 파밀리아, 구엘공원 – 가우디 건축과 지중해\n\n**3. 실용 팁**: 유레일패스(글로벌)를 구매하면 도시 간 이동이 편합니다. 환전은 최소화하고 체크카드(트래블월렛 등)를 주력으로 사용하세요. 소매치기 주의 – 크로스백 필수입니다."
    },
    {
        "instruction": "가족 여행으로 동남아 추천해줘",
        "input": "",
        "output": "당신은 전문 여행 컨설턴트입니다.\n\n**1. 상황 파악**: 가족 여행은 비행시간, 안전성, 아이 친화적 시설이 핵심 고려사항입니다.\n\n**2. 추천 여행지**:\n- **태국 방콕+파타야** (5일): 사파리월드, 놀이공원, 워터파크 – 비행 5.5시간, 가성비 최고\n- **베트남 다낭** (4일): 바나힐(테마파크), 미케비치, 호이안 – 비행 4.5시간, 리조트형 가족 여행\n- **싱가포르** (4일): 유니버셜 스튜디오, 가든스바이더베이, 센토사 – 치안 우수, 깔끔한 도시\n\n**3. 실용 팁**: 아이 동반 시 직항편을 추천하며, 현지 음식이 맞지 않을 수 있으니 간단한 비상식(컵라면, 과자)을 준비하세요. 여행자 보험은 가족 단체 상품이 개별보다 저렴합니다."
    },
    {
        "instruction": "혼자 여행하기 좋은 곳 추천",
        "input": "",
        "output": "당신은 전문 여행 컨설턴트입니다.\n\n**1. 상황 파악**: 혼자 여행은 치안, 대중교통 편의성, 혼밥/혼술 문화가 중요합니다.\n\n**2. 추천 여행지**:\n- **일본 도쿄·오사카**: 혼밥 문화 최고, 대중교통 완벽, 치안 우수 – 첫 솔로여행에 추천\n- **대만 타이베이**: 야시장 혼밥 천국, 물가 저렴, 한국어 메뉴 많음\n- **포르투갈 리스본**: 유럽 중 물가 저렴, 따뜻한 현지인, 혼행 커뮤니티 활발\n\n**3. 실용 팁**: 숙소는 게스트하우스나 호스텔(개인실)을 추천합니다 – 다른 여행자와 정보를 교환할 수 있습니다. 구글맵에 오프라인 지도를 미리 다운로드하고, 숙소 주소는 현지어로도 저장해두세요."
    },
    {
        "instruction": "여행 가방 꾸리는 팁 알려줘",
        "input": "",
        "output": "당신은 전문 여행 컨설턴트입니다.\n\n**1. 상황 파악**: 가방 꾸리기는 여행의 편의성을 결정하는 중요한 준비 과정입니다.\n\n**2. 핵심 팁**:\n- **의류 정리**: 돌돌 말기(rolling) 방식이 주름도 줄이고 공간 효율이 높습니다. 파킹큐브를 사용하면 정리가 더 쉽습니다.\n- **필수 vs 선택 구분**: 3일 이상 여행이면 세탁을 전제로 의류를 최소화하세요. 속옷 3벌 + 상의 3벌 + 하의 2벌이 기본입니다.\n- **기내 반입 주의**: 액체류는 100ml 이하 용기에 담아 1L 지퍼백 1개까지. 보조배터리는 반드시 기내 반입.\n\n**3. 실용 팁**: 무게를 줄이려면 짐의 절반을 싸고 나서 다시 절반을 빼보세요. 현지에서 사는 게 더 싸고 편한 물건(우산, 슬리퍼 등)은 과감히 빼세요."
    },
    {
        "instruction": "해외여행 보험 필요해?",
        "input": "",
        "output": "당신은 전문 여행 컨설턴트입니다.\n\n**1. 상황 파악**: 해외여행 보험은 선택이 아닌 필수입니다. 해외 의료비는 한국과 비교할 수 없을 정도로 비쌉니다.\n\n**2. 주요 보장 항목**:\n- **해외 의료비**: 미국은 응급실 방문만으로 수백만 원. 보험 없으면 큰 부담\n- **여행 취소/지연**: 항공편 지연, 수하물 분실 시 보상\n- **배상 책임**: 여행 중 타인의 재물을 손상했을 때 보상\n\n**3. 실용 팁**: 신용카드 자동 부가 보험이 있는지 먼저 확인하세요. 별도 가입 시 온라인(삼성화재, DB손해보험 등)이 오프라인보다 30~50% 저렴합니다. 기존 질환이 있다면 고지 의무를 확인하세요."
    },
    {
        "instruction": "비행기 싼 거 어떻게 구해?",
        "input": "",
        "output": "당신은 전문 여행 컨설턴트입니다.\n\n**1. 상황 파악**: 항공권은 여행 예산의 가장 큰 비중을 차지하며, 구매 타이밍과 방법에 따라 가격 차이가 큽니다.\n\n**2. 저가 항공권 확보 전략**:\n- **비교 검색**: 스카이스캐너, 구글 플라이트, 네이버 항공권에서 교차 비교\n- **구매 타이밍**: 출발 6~8주 전이 국제선 최적 구매 시점. 화~목요일이 가격이 낮은 편\n- **유연한 일정**: '전체 월' 검색으로 가장 저렴한 날짜를 먼저 찾고 일정을 맞추세요\n- **LCC 활용**: 진에어, 에어부산, 피치항공 등 저비용항공사의 프로모션 알림 설정\n\n**3. 실용 팁**: 시크릿 모드(프라이빗 브라우징)로 검색하면 쿠키 기반 가격 인상을 피할 수 있습니다. 경유편이 직항보다 20~40% 저렴한 경우가 많으니 시간 여유가 있다면 고려해보세요."
    },
    {
        "instruction": "여름에 시원한 여행지 추천",
        "input": "",
        "output": "당신은 전문 여행 컨설턴트입니다.\n\n**1. 상황 파악**: 한국의 여름(7~8월)은 고온다습하므로, 시원한 여행지를 찾는 수요가 많습니다.\n\n**2. 추천 여행지**:\n- **국내**: 강원도 평창·대관령(해발 700m, 서울보다 5~7℃ 낮음), 울릉도(해양성 기후로 선선)\n- **해외 근거리**: 홋카이도 삿포로(여름 평균 22℃, 라벤더 시즌), 블라디보스토크(서늘한 러시아 극동)\n- **해외 장거리**: 뉴질랜드(남반구 겨울, 스키 가능), 북유럽 노르웨이·핀란드(백야 체험)\n\n**3. 실용 팁**: 국내 강원도는 성수기 숙소가 빨리 마감되니 2개월 전 예약을 추천합니다. 해외의 경우 홋카이도 직항(대한항공, 진에어)이 비행 2.5시간으로 가장 접근성이 좋습니다."
    },
    {
        "instruction": "신혼여행 어디로 갈까?",
        "input": "",
        "output": "당신은 전문 여행 컨설턴트입니다.\n\n**1. 상황 파악**: 신혼여행은 로맨틱한 분위기, 프라이빗한 공간, 특별한 경험이 핵심입니다.\n\n**2. 추천 여행지** (예산별):\n- **프리미엄**: 몰디브(수상빌라, 올인클루시브), 산토리니(그리스 에게해 석양) – 7일 기준 500~800만 원\n- **가성비**: 발리(풀빌라가 합리적), 다낭(리조트+관광 밸런스) – 7일 기준 200~400만 원\n- **특별 경험**: 하와이(액티비티 풍부), 스위스(기차 여행+알프스) – 7일 기준 400~600만 원\n\n**3. 실용 팁**: 신혼여행 전문 여행사를 통하면 호텔 업그레이드, 허니문 패키지 등 혜택을 받을 수 있습니다. 시기는 비수기(5~6월, 9~10월)가 가격 대비 만족도가 높습니다."
    },
    {
        "instruction": "일본 온천 여행 추천해줘",
        "input": "",
        "output": "당신은 전문 여행 컨설턴트입니다.\n\n**1. 상황 파악**: 일본 온천(온센)은 지역마다 특색이 다르며, 효능과 분위기가 다양합니다.\n\n**2. 추천 온천 지역**:\n- **벳부** (오이타현): 일본 최대 온천 도시, 지옥순례 관광, 모래찜질 체험 – 후쿠오카에서 기차 2시간\n- **하코네** (가나가와현): 도쿄 근교(1.5시간), 후지산 조망, 고급 료칸 다수\n- **구사쓰** (군마현): 자연 용출량 일본 1위, 유바타케(탕밭) 야경이 유명\n\n**3. 실용 팁**: 료칸(전통 여관)은 1박 2식(조식+석식) 포함이 일반적이며, 가이세키 요리를 즐길 수 있습니다. 문신이 있으면 입장 제한이 있는 곳이 많으니, '타투 OK' 온천을 미리 확인하세요."
    },
    {
        "instruction": "배낭여행 준비물 알려줘",
        "input": "",
        "output": "당신은 전문 여행 컨설턴트입니다.\n\n**1. 상황 파악**: 배낭여행은 짐을 최소화하면서도 필수 아이템을 빠짐없이 챙기는 것이 핵심입니다.\n\n**2. 카테고리별 준비물**:\n- **필수 서류**: 여권(유효기간 6개월 이상), 여행자 보험증서, 숙소·항공 예약 확인서(오프라인 프린트)\n- **가방·수납**: 40~50L 배낭(기내 반입 가능 사이즈), 서브백(데이팩), 방수 파우치\n- **의류**: 속건성 소재 3세트, 바람막이, 경량 패딩(압축 가능), 편한 운동화\n- **전자기기**: 멀티 어댑터, 보조배터리(20000mAh), 스마트폰 방수팩\n- **위생·건강**: 개인 세면도구(소분), 상비약(소화제, 진통제, 밴드), 자외선 차단제\n\n**3. 실용 팁**: 짐 무게는 체중의 20% 이하가 적정합니다. 출발 전 배낭을 메고 30분 걸어보며 무게를 체크하세요."
    },
    {
        "instruction": "아이와 함께 갈 만한 국내 여행지",
        "input": "",
        "output": "당신은 전문 여행 컨설턴트입니다.\n\n**1. 상황 파악**: 아이 동반 여행은 이동 거리, 체험 프로그램, 편의시설(유모차·수유실)이 중요합니다.\n\n**2. 연령별 추천**:\n- **영유아(0~3세)**: 제주 아쿠아플라넷, 경주 보문단지(넓은 산책로), 가평 아침고요수목원 – 유모차 접근성 좋은 곳\n- **유아(4~7세)**: 에버랜드·롯데월드(놀이공원), 국립과천과학관(체험형), 보령 머드축제(여름)\n- **초등(8세~)**: 경주 역사탐방(첨성대·불국사), 강원 레일바이크, 전주 한옥마을 한복 체험\n\n**3. 실용 팁**: 이동 시간 2시간 이내로 짜는 것이 아이 컨디션 관리에 좋습니다. 숙소는 키즈풀이나 놀이방이 있는 리조트(대명, 한화 등)를 추천합니다."
    },
    {
        "instruction": "제주도 맛집 추천해줘",
        "input": "",
        "output": "당신은 전문 여행 컨설턴트입니다.\n\n**1. 상황 파악**: 제주도는 흑돼지, 해산물, 감귤 디저트 등 특색 있는 먹거리가 풍부합니다.\n\n**2. 카테고리별 추천**:\n- **흑돼지**: 돈사돈(서귀포), 제주 흑돈가(제주시) – 고기 질이 좋고 가성비 우수\n- **해산물**: 올레해장국(성게국), 해녀의집(회·뚝배기) – 현지인 맛집\n- **카페·디저트**: 카페 델문도(바다뷰), 우유부단(감귤라떼) – 인스타 감성\n- **현지식**: 고기국수(올래국수), 빙떡(시장 간식) – 제주 전통 맛\n\n**3. 실용 팁**: 유명 맛집은 오전 11시 전에 도착해야 웨이팅 없이 입장 가능합니다. 네이버 지도의 '제주 맛집' 저장 기능을 활용해 동선별로 미리 정리하세요."
    },
    {
        "instruction": "여행 중 환전 어떻게 해?",
        "input": "",
        "output": "당신은 전문 여행 컨설턴트입니다.\n\n**1. 상황 파악**: 환전 방법에 따라 같은 금액도 수수료 차이가 크게 날 수 있습니다.\n\n**2. 환전 방법 비교**:\n- **은행 방문**: 주거래 은행에서 환전 시 우대 환율 적용(50~90% 우대). 사전 예약 필수\n- **공항 환전**: 가장 편리하지만 환율이 불리. 급할 때만 소액 환전용으로 사용\n- **트래블월렛/와이즈**: 체크카드형으로 해외 ATM 인출 + 결제 가능. 환율 우대가 가장 좋음\n- **현지 ATM**: 도착 후 ATM에서 인출 – 수수료 확인 필수(1회 3,000~5,000원)\n\n**3. 실용 팁**: 현금은 여행 경비의 30% 정도만 환전하고 나머지는 카드로 결제하세요. 일본은 아직 현금 문화가 강하니 넉넉히, 유럽은 카드 위주로 준비하면 됩니다."
    },
    {
        "instruction": "캠핑 처음인데 뭐 준비해야 해?",
        "input": "",
        "output": "당신은 전문 여행 컨설턴트입니다.\n\n**1. 상황 파악**: 캠핑 초보자는 장비 투자를 최소화하면서 핵심 장비를 갖추는 것이 중요합니다.\n\n**2. 필수 장비 리스트**:\n- **텐트**: 2~3인용 돔텐트(설치 쉬움). 처음엔 렌탈(캠핑나라 등)로 시작 추천\n- **침구**: 침낭 + 에어매트(바닥 냉기 차단 필수)\n- **조리**: 버너 + 코펠 세트, 아이스박스(음식 보관)\n- **조명**: LED 랜턴 2개(메인 + 텐트 내부용)\n- **기타**: 타프(비·햇빛 차단), 접이식 테이블·의자, 망치(펙 고정용)\n\n**3. 실용 팁**: 첫 캠핑은 시설이 잘 갖춰진 오토캠핑장(화장실·개수대·전기)을 추천합니다. 가평, 춘천, 태안 지역에 초보자 친화적 캠핑장이 많습니다."
    },
    {
        "instruction": "서울 관광 코스 추천해줘",
        "input": "",
        "output": "당신은 전문 여행 컨설턴트입니다.\n\n**1. 상황 파악**: 서울은 역사·문화·쇼핑·맛집이 밀집된 도시로, 테마별로 코스를 짜면 효율적입니다.\n\n**2. 테마별 추천 코스**:\n- **역사 코스**: 경복궁(수문장 교대식) → 북촌 한옥마을 → 인사동(전통 공예·찻집) → 광장시장(야식)\n- **힙 코스**: 성수동(카페·팝업스토어) → 뚝섬 서울숲 → 한남동(편집숍) → 이태원(세계 음식)\n- **야경 코스**: N서울타워(남산 케이블카) → 반포 달빛무지개분수 → 여의도 한강공원(치맥)\n\n**3. 실용 팁**: 서울 지하철은 T-money 카드로 환승 할인 받으세요. 1일 교통카드(기후동행카드)를 이용하면 교통비를 크게 절약할 수 있습니다."
    },
    {
        "instruction": "비 오는 날 여행지 추천",
        "input": "",
        "output": "당신은 전문 여행 컨설턴트입니다.\n\n**1. 상황 파악**: 비 오는 날에도 충분히 즐길 수 있는 실내 관광지와 '비가 와서 더 좋은' 장소가 있습니다.\n\n**2. 추천 여행지**:\n- **실내 관광**: 국립중앙박물관(서울), 아쿠아리움(코엑스·제주), 키자니아(아이 동반) – 날씨 무관\n- **비가 운치 있는 곳**: 담양 죽녹원(대나무숲 빗소리), 전주 한옥마을(처마 밑 빗방울), 남이섬(안개 속 메타세콰이어 길)\n- **체험형**: 도자기 만들기(이천), 쿠킹클래스(서울), 와인 테이스팅(영동) – 실내에서 특별한 경험\n\n**3. 실용 팁**: 우비와 방수 가방커버를 준비하면 비 오는 날도 야외 활동이 가능합니다. 박물관·미술관은 비 오는 날 방문객이 적어 오히려 여유롭게 감상할 수 있습니다."
    },
    {
        "instruction": "해외에서 인터넷 어떻게 써?",
        "input": "",
        "output": "당신은 전문 여행 컨설턴트입니다.\n\n**1. 상황 파악**: 해외 인터넷은 포켓와이파이, 유심, eSIM 중 여행 스타일에 맞게 선택합니다.\n\n**2. 방법별 비교**:\n- **포켓와이파이**: 여러 기기 동시 연결 가능, 그룹 여행에 적합 – 1일 5,000~10,000원. 충전·반납 번거로움\n- **현지 유심**: 현지 번호 부여, 통화 가능, 개인 여행에 적합 – 3~7일 10,000~20,000원. 기존 번호 사용 불가\n- **eSIM**: 물리적 교체 없이 앱으로 개통, 기존 번호 유지(듀얼심) – 에어알로, 유심사 등에서 구매\n\n**3. 실용 팁**: 최근에는 eSIM이 가장 편리합니다(아이폰 XS 이상, 갤럭시 S21 이상 지원). 출발 전 앱에서 미리 구매·설치하고, 도착 후 활성화만 하면 됩니다."
    },
    {
        "instruction": "베트남 다낭 여행 코스",
        "input": "",
        "output": "당신은 전문 여행 컨설턴트입니다.\n\n**1. 상황 파악**: 다낭은 해변 리조트와 역사 도시(호이안·후에)를 함께 즐길 수 있는 여행지입니다.\n\n**2. 추천 코스** (4박 5일):\n- **Day 1**: 미케비치 해변 + 한시장(현지 분위기) + 용다리 야경(주말 불쇼)\n- **Day 2**: 바나힐(케이블카+골든브릿지+테마파크) – 하루 코스\n- **Day 3**: 호이안 올드타운(세계문화유산) + 등불축제(음력 14일) + 코코넛 배\n- **Day 4**: 오행산(마블마운틴) + 시내 스파·마사지 + 루프탑 바\n- **Day 5**: 한강 카페거리 + 기념품 쇼핑 + 귀국\n\n**3. 실용 팁**: 그랩(Grab) 앱이 택시보다 안전하고 저렴합니다. 건기(2~7월)가 여행 적기이며, 10~11월은 태풍 시즌이니 피하세요."
    },
    {
        "instruction": "여행지에서 사진 잘 찍는 법",
        "input": "",
        "output": "당신은 전문 여행 컨설턴트입니다.\n\n**1. 상황 파악**: 스마트폰만으로도 여행 사진을 훨씬 잘 찍을 수 있는 핵심 원칙이 있습니다.\n\n**2. 핵심 촬영 팁**:\n- **골든아워 활용**: 해 뜨고 1시간, 해 지기 1시간 전의 빛이 가장 예쁩니다. 이 시간에 인물·풍경 촬영\n- **삼분할 법칙**: 카메라 격자선을 켜고, 피사체를 교차점에 배치하면 안정적인 구도\n- **역광 회피**: 태양을 등지고 찍거나, 실루엣을 의도적으로 활용. 역광 시 HDR 모드 켜기\n- **낮은 각도**: 건물·음식은 아래에서 위로, 인물은 살짝 위에서 아래로 찍으면 비율이 좋아짐\n\n**3. 실용 팁**: 같은 장소에서 3~5장을 다른 구도로 찍어두세요. 후보정은 VSCO나 라이트룸 모바일앱으로 밝기·대비·채도만 살짝 조정하면 충분합니다."
    },
    {
        "instruction": "가을 단풍 여행지 추천",
        "input": "",
        "output": "당신은 전문 여행 컨설턴트입니다.\n\n**1. 상황 파악**: 한국의 단풍 시즌은 9월 말 설악산에서 시작해 11월 초 남쪽까지 내려갑니다.\n\n**2. 추천 여행지** (시기순):\n- **9월 말~10월 초**: 설악산(케이블카에서 조망), 오대산 월정사 전나무숲\n- **10월 중순**: 내장산(단풍터널 전국 1위), 덕유산, 지리산 피아골\n- **10월 말~11월 초**: 경주 불국사(은행나무+단풍), 담양 메타세콰이어 가로수길, 해남 대흥사\n\n**3. 실용 팁**: 단풍 명소는 주말 교통 정체가 극심합니다. 평일 방문이 불가능하면 새벽 출발(오전 7시 도착 목표)이 최선입니다. 기상청 '단풍 예보'를 확인하고 일정을 잡으세요."
    },
    {
        "instruction": "호캉스 추천해줘",
        "input": "",
        "output": "당신은 전문 여행 컨설턴트입니다.\n\n**1. 상황 파악**: 호캉스는 호텔 시설(수영장, 스파, 다이닝)을 최대한 활용하는 것이 핵심입니다.\n\n**2. 추천 호텔** (서울/근교):\n- **럭셔리**: 시그니엘 서울(롯데타워 전망), 포시즌스 서울(루프탑 풀) – 1박 40~70만 원\n- **가성비 럭셔리**: 글래드 여의도, 노보텔 동대문 – 1박 15~25만 원, 수영장·조식 포함 패키지\n- **리조트형**: 파라다이스시티 인천(워터파크+카지노), 롤링힐스 화성(골프+스파) – 도심 탈출\n\n**3. 실용 팁**: 호텔 예약은 공식 웹사이트 > 야놀자/여기어때 > OTA(호텔스컴바인) 순으로 비교하세요. 일~목 체크인이 주말보다 30~50% 저렴하며, 레이트 체크아웃(보통 2시까지)을 미리 요청하면 여유롭습니다."
    },
    {
        "instruction": "렌터카 여행 팁",
        "input": "",
        "output": "당신은 전문 여행 컨설턴트입니다.\n\n**1. 상황 파악**: 렌터카 여행은 자유로운 동선이 장점이지만, 사전 준비가 중요합니다.\n\n**2. 핵심 팁**:\n- **예약**: 최소 2주 전 예약. 제주도는 롯데·SK·AJ렌터카가 안정적. 보험은 완전자차(CDW) 필수 가입\n- **차량 선택**: 2인은 경차/소형, 4인 가족은 중형 SUV 추천. 제주는 SUV가 인기(오름·비포장도로)\n- **인수 시**: 외관 흠집을 반드시 사진·영상 촬영하고 직원과 함께 확인. 블랙박스 작동 확인\n\n**3. 실용 팁**: 네이버 지도나 카카오맵의 '경유지 추가' 기능으로 동선을 미리 최적화하세요. 제주도는 일 평균 주행 100~150km를 넘지 않게 일정을 짜면 피로 없이 여행 가능합니다."
    },
    {
        "instruction": "여행 예산 세우는 법",
        "input": "",
        "output": "당신은 전문 여행 컨설턴트입니다.\n\n**1. 상황 파악**: 체계적인 예산 계획은 여행의 질을 높이고 예상 밖 지출을 방지합니다.\n\n**2. 예산 항목별 가이드**:\n- **항공**: 전체 예산의 25~35%. 비교 검색 후 가장 먼저 확정\n- **숙소**: 전체 예산의 25~30%. 위치(역 근처)와 조식 포함 여부 확인\n- **식비**: 전체 예산의 20~25%. 1일 3식 기준, 로컬 식당 : 레스토랑 = 2 : 1 비율 추천\n- **교통·입장**: 전체 예산의 10~15%. 교통패스·콤보 티켓으로 절약\n- **비상금**: 전체 예산의 10%. 예상치 못한 지출(의료, 분실 등) 대비\n\n**3. 실용 팁**: 구글 시트나 엑셀로 일자별 예산표를 만들어 실시간 지출을 기록하세요. '트리플' 앱의 가계부 기능도 편리합니다. 현지 물가는 Numbeo 사이트에서 미리 확인 가능합니다."
    },
    {
        "instruction": "스위스 여행 추천 코스",
        "input": "",
        "output": "당신은 전문 여행 컨설턴트입니다.\n\n**1. 상황 파악**: 스위스는 알프스 자연경관과 정밀한 기차 시스템이 매력인 나라입니다.\n\n**2. 추천 코스** (7일):\n- **Day 1-2 (인터라켄)**: 융프라우 등정(유럽의 지붕), 호수 유람, 패러글라이딩\n- **Day 3-4 (체르마트)**: 마터호른 조망, 고르너그라트 전망대, 하이킹\n- **Day 5 (루체른)**: 카펠교, 빈사의 사자상, 리기산 전망대\n- **Day 6-7 (베른+취리히)**: 수도 베른 구시가지(세계문화유산), 취리히 호수, 쇼핑\n\n**3. 실용 팁**: 스위스 트래블 패스(연속 4/8/15일)를 구매하면 기차·버스·유람선이 무제한이고 산악 열차 50% 할인입니다. 물가가 매우 비싸므로(맥도날드 세트 2만 원) 쿱(Coop) 슈퍼에서 식사를 해결하면 절약됩니다."
    },
    {
        "instruction": "크루즈 여행 어때?",
        "input": "",
        "output": "당신은 전문 여행 컨설턴트입니다.\n\n**1. 상황 파악**: 크루즈 여행은 이동과 숙박이 동시에 해결되는 독특한 여행 형태입니다.\n\n**2. 크루즈 여행의 장단점**:\n- **장점**: 짐 풀고 여러 도시를 편하게 방문, 선내 시설(수영장·쇼·뷔페) 포함, 가족·어르신 여행에 적합\n- **단점**: 기항지 체류 시간이 짧음(보통 6~8시간), 배멀미 가능성, 인터넷 비용 별도\n- **추천 노선**: 지중해(바르셀로나→나폴리→그리스), 알래스카(빙하 크루즈), 일본(후쿠오카→나가사키)\n\n**3. 실용 팁**: 배멀미는 중앙·하층 객실이 가장 적습니다. 보나민(멀미약)을 미리 준비하세요. 한국 출발 크루즈는 인천·부산항에서 출발하며, 코스타·MSC 크루즈가 가성비가 좋습니다."
    },
    {
        "instruction": "반려동물과 여행할 수 있는 곳",
        "input": "",
        "output": "당신은 전문 여행 컨설턴트입니다.\n\n**1. 상황 파악**: 반려동물 동반 여행은 시설의 펫 허용 여부를 사전에 꼼꼼히 확인하는 것이 핵심입니다.\n\n**2. 추천 여행지·숙소**:\n- **강원도 양양·속초**: 해변 산책 가능, 펫 펜션 다수(스테이 펫, 엘리펫 등)\n- **경기도 가평**: 자라섬, 남이섬(소형견 입장 가능), 펫 글램핑 시설 풍부\n- **제주도**: 이호테우해변(반려견 해변), 서귀포 펫카페, 펫 동반 올레길\n\n**3. 실용 팁**: 숙소 예약 시 '펫 동반' 필터를 꼭 확인하고, 추가 요금(보통 1~3만 원)과 크기 제한을 체크하세요. 차량 이동 시 펫 캐리어나 안전벨트를 사용하고, 30분마다 휴식을 취해주세요."
    },
    {
        "instruction": "워케이션 좋은 곳 추천",
        "input": "",
        "output": "당신은 전문 여행 컨설턴트입니다.\n\n**1. 상황 파악**: 워케이션(Work + Vacation)은 안정적인 인터넷, 작업 공간, 여행 즐거움의 균형이 핵심입니다.\n\n**2. 추천 워케이션 지역**:\n- **제주도**: 제주창조경제혁신센터(무료 코워킹), 카페 작업 환경 최고, 퇴근 후 바다 산책\n- **양양·강릉**: 서퍼들의 성지, 바다뷰 카페에서 작업, 서핑·산책으로 리프레시\n- **해외 – 치앙마이(태국)**: 디지털 노마드 성지, 코워킹 스페이스 풍부, 월 생활비 100만 원 이하\n\n**3. 실용 팁**: 숙소는 '장기 할인'이 있는 에어비앤비나 한달살기 플랫폼(살아보기, 먼슬리)을 이용하세요. Wi-Fi 속도(최소 50Mbps)와 모니터·의자 제공 여부를 반드시 확인하세요."
    },
]

print(f"Version B 데이터 수: {len(version_b_data)}")
print(f"\n[예시] Version B 첫 번째 데이터:")
print(json.dumps(version_b_data[0], ensure_ascii=False, indent=2))

### Q1. 두 버전의 데이터 품질 차이를 분석하세요.

아래 코드를 실행하여 두 버전의 데이터 특성을 비교하고, 차이점을 정리하세요.

In [ ]:
# Q1: 두 버전의 데이터 특성 비교
def analyze_dataset(data, name):
    output_lengths = [len(d["output"]) for d in data]
    has_structure = sum(1 for d in data if "**1." in d["output"] or "**2." in d["output"])
    has_role = sum(1 for d in data if "컨설턴트" in d["output"] or "튜터" in d["output"])

    print(f"\n{'='*50}")
    print(f"  {name} 분석")
    print(f"{'='*50}")
    print(f"  데이터 수: {len(data)}")
    print(f"  평균 답변 길이: {np.mean(output_lengths):.0f} 글자")
    print(f"  최소/최대 답변 길이: {min(output_lengths)} / {max(output_lengths)} 글자")
    print(f"  구조적 답변 비율: {has_structure/len(data)*100:.0f}%")
    print(f"  역할 부여 비율: {has_role/len(data)*100:.0f}%")

analyze_dataset(version_a_data, "Version A (Low Quality)")
analyze_dataset(version_b_data, "Version B (High Quality)")

In [ ]:
# 답변 길이 분포 시각화
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

a_lengths = [len(d["output"]) for d in version_a_data]
b_lengths = [len(d["output"]) for d in version_b_data]

axes[0].hist(a_lengths, bins=15, color='salmon', edgecolor='black', alpha=0.7)
axes[0].set_title('Version A: Output Length Distribution', fontsize=13)
axes[0].set_xlabel('Character Count')
axes[0].set_ylabel('Frequency')
axes[0].axvline(np.mean(a_lengths), color='red', linestyle='--', label=f'Mean: {np.mean(a_lengths):.0f}')
axes[0].legend()

axes[1].hist(b_lengths, bins=15, color='skyblue', edgecolor='black', alpha=0.7)
axes[1].set_title('Version B: Output Length Distribution', fontsize=13)
axes[1].set_xlabel('Character Count')
axes[1].set_ylabel('Frequency')
axes[1].axvline(np.mean(b_lengths), color='blue', linestyle='--', label=f'Mean: {np.mean(b_lengths):.0f}')
axes[1].legend()

plt.tight_layout()
plt.show()

---
## 실습 2: LoRA 파인튜닝 A/B 테스트

동일한 베이스 모델(EXAONE-4.0-1.2B)에 동일한 LoRA 설정으로, **데이터만 다르게** 파인튜닝합니다.

> 💡 1.2B 모델은 fp16으로도 T4 GPU에 충분히 올라가므로, 4-bit 양자화 없이 fp16 + LoRA로 학습합니다.

| 항목 | Version A | Version B |
|------|-----------|----------|
| 베이스 모델 | EXAONE-4.0-1.2B | EXAONE-4.0-1.2B |
| 정밀도 | fp16 (양자화 없음) | fp16 (양자화 없음) |
| LoRA r / alpha | 16 / 32 | 16 / 32 |
| target_modules | q_proj, v_proj | q_proj, v_proj |
| max epochs | 100 (Early Stopping, patience=5) | 100 (Early Stopping, patience=5) |
| batch size | 8 | 8 |
| max_length | 1024 | 1024 |
| LR scheduler | Cosine w/ Warm Restarts (3 cycles) | Cosine w/ Warm Restarts (3 cycles) |
| **데이터** | **Low Quality (25 train / 5 eval)** | **High Quality (25 train / 5 eval)** |

### Q2. EXAONE-4.0-1.2B 모델을 불러오고 학습하는 코드를 작성하세요.

In [ ]:
# =============================================================
# 베이스 모델 및 토크나이저 로드 (fp16 — 1.2B 모델은 양자화 불필요)
# =============================================================
MODEL_NAME = "LGAI-EXAONE/EXAONE-4.0-1.2B"

# A2.


### Q3. Version A, B 데이터로 각각 다른 2개의 모델을 따로 파인튜닝을 실행하세요. (약 5~10분 소요)

In [ ]:
# A3.


### Q4. 두 모델의 학습 Loss 곡선을 비교하세요.

In [ ]:
# A4.


---
## 실습 3: A/B 추론 비교

학습된 두 모델에 동일한 10개 평가 프롬프트를 넣고, 출력 품질을 비교합니다.

In [ ]:
# =============================================================
# 평가 프롬프트 10개 정의
# =============================================================
eval_prompts = [
    "3박 4일 도쿄 여행 코스를 짜줘",
    "예산 50만원으로 갈 수 있는 해외 여행지 추천해줘",
    "겨울 제주도 여행에서 꼭 해야 할 것들 알려줘",
    "여행 중 짐을 분실했을 때 어떻게 해야 해?",
    "유럽 여행 시 유레일패스가 정말 필요한지 알려줘",
    "아이 둘 데리고 괌 여행 갈 건데 팁 좀 줘",
    "국내 섬 여행지 중에서 추천할 곳 있어?",
    "여행 사진을 예쁘게 편집하는 앱 추천해줘",
    "혼자 배낭여행 중 외로울 때 어떻게 해?",
    "비행기에서 잠 잘 자는 방법 알려줘",
]

print(f"평가 프롬프트 {len(eval_prompts)}개 준비 완료")
for i, p in enumerate(eval_prompts):
    print(f"  {i+1}. {p}")

### Q5. 추론 함수를 정의하고, 두 모델의 추론 결과를 비교하세요. (각 약 3~5분 소요)

In [ ]:
# A5.


### Q6. 수동 평가: 각 프롬프트에 대해 A/B 답변의 품질을 1~5점으로 채점하세요.

**채점 기준:**
| 점수 | 기준 |
|------|------|
| 1점 | 질문과 무관한 답변, 의미 없는 반복 |
| 2점 | 질문과 관련은 있으나 정보가 매우 부족 |
| 3점 | 기본적인 답변은 하지만 구조/깊이 부족 |
| 4점 | 유용한 정보를 구조적으로 제공 |
| 5점 | 전문가 수준의 체계적이고 실용적인 답변 |

In [ ]:
# Q6: 수동 평가 프레임워크
# TODO: 각 프롬프트에 대해 Version A, B의 점수를 입력하세요 (1~5점)
scores = {
    "prompt": eval_prompts,
    "version_a_score": [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],  # TODO: 채점 결과 입력
    "version_b_score": [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],  # TODO: 채점 결과 입력
}

df_scores = pd.DataFrame(scores)
df_scores.index = range(1, len(eval_prompts) + 1)
df_scores.index.name = "번호"

# 통계 요약
print("="*60)
print("  A/B 평가 결과 요약")
print("="*60)
print(f"  Version A 평균 점수: {df_scores['version_a_score'].mean():.2f}")
print(f"  Version B 평균 점수: {df_scores['version_b_score'].mean():.2f}")
print(f"  Version B 우위 프롬프트 수: {(df_scores['version_b_score'] > df_scores['version_a_score']).sum()}/10")

# 시각화
fig, ax = plt.subplots(figsize=(12, 5))
x = np.arange(1, 11)
width = 0.35

bars_a = ax.bar(x - width/2, df_scores['version_a_score'], width,
                label='Version A (Low Quality)', color='salmon', alpha=0.8)
bars_b = ax.bar(x + width/2, df_scores['version_b_score'], width,
                label='Version B (High Quality)', color='skyblue', alpha=0.8)

ax.set_xlabel('Prompt Number', fontsize=12)
ax.set_ylabel('Score (1-5)', fontsize=12)
ax.set_title('Manual Evaluation: Version A vs Version B', fontsize=14)
ax.set_xticks(x)
ax.set_ylim(0, 5.5)
ax.legend()
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

---
## 실습 4: Format Mismatch 실험

학습 시 사용한 포맷과 **다른 포맷**으로 추론하면 어떻게 되는지 실험합니다.

이 실험의 핵심 포인트:
> **"데이터 포맷 align을 안 하면, 학습이 잘 되어도 inference에서 깨진다"**

In [ ]:
# =============================================================
# Format Mismatch 실험
# =============================================================
test_prompt = "제주도 3박4일 가족 여행 코스를 짜줘"

# 1) 학습과 동일한 포맷 (Matched) - EXAONE chat template
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": test_prompt},
]
matched_format = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

# 2) 다른 포맷들 (Mismatched)
mismatched_formats = {
    "plain_text": test_prompt,  # 포맷 없이 자연어만
    "alpaca_style": f"### Instruction:\n{test_prompt}\n\n### Input:\n\n\n### Response:\n",  # Alpaca 스타일
    "chat_style": f"User: {test_prompt}\nAssistant:",  # 일반 채팅 스타일
    "chatml": f"<|im_start|>user\n{test_prompt}<|im_end|>\n<|im_start|>assistant\n",  # ChatML (Qwen 등)
}

print("포맷 비교:")
print(f"\n[Matched Format - EXAONE Chat Template]:\n{matched_format}")
for name, fmt in mismatched_formats.items():
    print(f"\n[Mismatched - {name}]:\n{fmt}")

In [ ]:
# Version B 모델로 포맷별 추론 실행
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True,
)
model = PeftModel.from_pretrained(model, "./model_version_b")
model.eval()

def generate_response(prompt_text, max_tokens=300):
    inputs = tokenizer(prompt_text, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_tokens,
            temperature=0.1,  # EXAONE 1.2B 한국어: 낮은 temperature 권장
            top_p=0.9,
            do_sample=True,
            repetition_penalty=1.2,
            pad_token_id=tokenizer.pad_token_id,
        )
    return tokenizer.decode(outputs[0][inputs["input_ids"].shape[1]:], skip_special_tokens=True).strip()

separator = "=" * 70

# Matched 포맷 추론 (EXAONE chat template)
print(separator)
print("  MATCHED FORMAT (EXAONE chat template - 학습 포맷과 동일)")
print(separator)
matched_result = generate_response(matched_format)
print(matched_result[:500])

# Mismatched 포맷 추론
for name, fmt in mismatched_formats.items():
    print(f"\n{separator}")
    print(f"  MISMATCHED FORMAT: {name}")
    print(separator)
    result = generate_response(fmt)
    print(result[:500])

del model
torch.cuda.empty_cache()

### Q7. Format Mismatch 실험 결과를 분석하세요.

아래 질문에 답하세요:

1. **Matched 포맷과 Mismatched 포맷의 출력 차이**는 어떤가요?
2. 어떤 Mismatched 포맷이 가장 **성능 저하가 심한**가요? 그 이유는?
3. Alpaca 스타일(`### Instruction:`)이나 ChatML 포맷(`<|im_start|>`)으로 넣었을 때의 결과는 어떤가요? (EXAONE 모델의 고유 chat template과 관련지어 생각해보세요)
4. 이 실험에서 얻은 교훈을 한 문장으로 정리하세요.

> **핵심 교훈**: _______________________________________________

In [ ]:
# A7: 직접 분석 결과를 아래에 작성해보세요. 어떤 점이 크게 다르다고 생각하나요?
"""
TODO: 위 질문에 대한 답변을 작성하세요

1. Matched vs Mismatched 차이:


2. 가장 성능 저하가 심한 포맷과 이유:


3. Alpaca/ChatML 포맷 결과 분석 (EXAONE chat template과의 차이):


4. 핵심 교훈:

"""